# Carga Silver - Stage Cotação Histórica

Este notebook realiza a carga na silver cotação histórica (stg_cotacao_historica) a partir dos dados da tabela bronze cotação histórica (raw_cotahist).

## Imports

In [15]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, col, trim, current_timestamp
from pyspark.sql.types import DecimalType, IntegerType

## Start Spark Session

In [16]:
# Inicializa uma SparkSession
builder = SparkSession.builder \
    .appName("Leitura e Transformação Delta Table") \
    .master("local[*]") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

print(f"\n✅ Spark {spark.version} iniciado!\n")


✅ Spark 3.5.7 iniciado!



## Variables

In [17]:
# Define caminhos locais onde serão armazenadas as tabelas Delta
base_bronze_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/bronze"
base_silver_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/silver"

# Define o caminho da tabela Delta Source
delta_path_cotahist = f"{base_bronze_path}/raw_cotahist"

# Define o caminho para salvar a tabela Delta transformada
delta_path_cotacao_historica = f"{base_silver_path}/stg_cotacao_historica"

## Read Table

In [ ]:
# Lendo a tabela Delta no DataFrame do PySpark
df_bronze = spark.read.format("delta").load(delta_path_cotahist)

print(f"[SUCESSO] Leitura da tabela Delta em: {delta_path_cotahist}")
print(f"Total de registros: {df_bronze.count()}")

[SUCESSO] Leitura da tabela Delta em: D:/Projetos/Jornada_financas_pessoais/data/delta/bronze/raw_cotahist
Total de registros: 5126223


## Transformation

In [ ]:
df_silver = df_bronze.select(
    # Campos String
    col("tipo_registro").cast("string").alias("tp_registro"),
    to_date(col("data_pregao"), "yyyyMMdd").alias("dt_pregao"),
    col("codigo_bdi").cast("string").alias("cd_bdi"),
    trim(col("codigo_negociacao")).alias("cd_negociacao"),
    col("tipo_mercado").cast("string").alias("tp_mercado"),
    trim(col("nome_resumido_empresa")).alias("nm_empresa"),
    trim(col("especificacao_papel")).alias("ds_especificacao_papel"),
    col("prazo_dias_mercado").cast("string").alias("nr_prazo_dias_mercado"),
    col("moeda_referencia").cast("string").alias("cd_moeda_referencia"),
    
    # Campos Decimais com precisão 11,2
    (col("preco_abertura_papel") / 100).cast(DecimalType(11,2)).alias("vl_abertura"),
    (col("preco_maximo_papel") / 100).cast(DecimalType(11,2)).alias("vl_maximo"),
    (col("preco_minimo_papel") / 100).cast(DecimalType(11,2)).alias("vl_minimo"),
    (col("preco_medio_papel") / 100).cast(DecimalType(11,2)).alias("vl_medio"),
    (col("preco_ultimo_negocio") / 100).cast(DecimalType(11,2)).alias("vl_ultimo_negocio"),
    (col("preco_melhor_oferta_compra") / 100).cast(DecimalType(11,2)).alias("vl_melhor_oferta_compra"),
    (col("preco_melhor_oferta_venda") / 100).cast(DecimalType(11,2)).alias("vl_melhor_oferta_venda"),
    
    # Campos Inteiros
    col("numero_negocios_efetuados").cast(IntegerType()).alias("qt_negocios_efetuados"),
    col("quantidade_total_titulos").cast(IntegerType()).alias("qt_total_titulos"),
    
    # Volume com precisão 16,2 (mantido conforme especificação)
    (col("volume_total_titulos") / 100).cast(DecimalType(16,2)).alias("vl_total_titulos"),
    
    # Demais campos
    (col("preco_exercicio_opcoes") / 100).cast(DecimalType(11,2)).alias("vl_exercicio_opcoes"),
    col("indicador_correcao_precos").cast("string").alias("cd_indicador_correcao"),
    to_date(col("data_vencimento_opcoes"), "yyyyMMdd").alias("dt_vencimento_opcoes"),
    col("fator_cotacao_papel").cast("string").alias("cd_fator_cotacao"),
    (col("preco_exercicio_pontos") / 1000000).cast(DecimalType(7,6)).alias("vl_exercicio_pontos"),
    col("codigo_papel_sistema").cast("string").alias("cd_papel_sistema"),
    col("numero_distribuicao_papel").cast("string").alias("nr_distribuicao_papel"),
    
    # Adiciona timestamp de inserção
    current_timestamp().alias("ts_insercao")
)

In [ ]:
# Salva o DataFrame transformado em formato Delta
df_silver.write.format("delta").mode("overwrite").option("overwriteSchema", "false").save(delta_path_cotacao_historica)

print(f"[SUCESSO] DataFrame transformado gravado no Delta Lake em: {delta_path_cotacao_historica}")

[SUCESSO] DataFrame transformado gravado no Delta Lake em: D:/Projetos/Jornada_financas_pessoais/data/delta/silver/stg_cotacao_historica


## Stop Spark Session

In [21]:
# Encerra a SparkSession
spark.stop()